In [1]:
import pdfplumber  # La librairie clé pour extraire texte et tableaux
import re
from pathlib import Path
from tqdm import tqdm
import pickle
import numpy as np

# --- Librairies pour le coeur du pipeline RAG ---
from langchain.text_splitter import RecursiveCharacterTextSplitter
# L'import peut varier selon la version de langchain
try:
    from langchain_core.documents import Document
except ImportError:
    from langchain.schema import Document
from sentence_transformers import SentenceTransformer
import faiss

print("✅ Librairies importées avec succès.")

# --- 1. Définir la structure des répertoires du projet ---
# Trouve la racine du projet de manière robuste
chemin_actuel = Path.cwd()
if chemin_actuel.name == "notebooks":
    RACINE_PROJET = chemin_actuel.parent
else:
    RACINE_PROJET = chemin_actuel

CHEMIN_DONNEES_BRUTES = RACINE_PROJET / "data" / "raw"
CHEMIN_DONNEES_TRAITEES = RACINE_PROJET / "data" / "processed"

# S'assurer que le dossier pour les données traitées existe
CHEMIN_DONNEES_TRAITEES.mkdir(parents=True, exist_ok=True)

print(f"-> Racine du projet définie sur : {RACINE_PROJET}")
print(f"-> Les PDF bruts seront lus depuis : {CHEMIN_DONNEES_BRUTES}")
print(f"-> L'index et les segments traités seront sauvegardés dans : {CHEMIN_DONNEES_TRAITEES}")

# --- 2. Configuration ---
# Définir le modèle d'embedding que nous utiliserons
# paraphrase-multilingual-MiniLM-L12-v2 est un excellent choix pour le français.
NOM_MODELE_SENTENCE_EMBEDDING ='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' 

# --- NOUVEAU : Configuration pour le débogage visuel ---
# Mettre sur True pour activer l'affichage de la comparaison avant/après nettoyage.
# Mettre sur False pour le traitement normal et rapide.
AFFICHER_COMPARAISON_AVANT_APRES = False

# Limiter l'affichage aux N premières pages de chaque PDF pour ne pas inonder la console.
PAGES_A_COMPARER_PAR_PDF = 2
chunk_size=400   
chunk_overlap=80

✅ Librairies importées avec succès.
-> Racine du projet définie sur : /home/elyes/stage/pdf-RAG-project
-> Les PDF bruts seront lus depuis : /home/elyes/stage/pdf-RAG-project/data/raw
-> L'index et les segments traités seront sauvegardés dans : /home/elyes/stage/pdf-RAG-project/data/processed


In [2]:
# ==============================================================================
# === DÉBUT DE LA SECTION AMÉLIORÉE : Fonctions de traitement de PDF ===
# ==============================================================================

def nettoyer_texte(texte: str) -> str:
    """
    Nettoie une chaîne de caractères pour la préparation à l'embedding.
    - Supprime les espaces multiples et les sauts de ligne superflus.
    - Corrige les césures de mots en fin de ligne (ex: "docu-\nment" -> "document").
    - Rejoint les lignes d'un même paragraphe.
    - Supprime les lignes entièrement vides.
    """
    if not texte:
        return ""
        
    # 1. Corriger les césures de mots en fin de ligne
    texte = re.sub(r'(\w+)-\n(\w+)', r'\1\2', texte)
    
    # 2. Remplacer les sauts de ligne multiples par un double saut de ligne (marqueur de paragraphe)
    texte = re.sub(r'\n\s*\n', '\n\n', texte)
    
    # 3. Supprimer les sauts de ligne simples qui ne sont pas précédés par un point ou dans une liste.
    # Cela permet de joindre les phrases coupées par un retour à la ligne.
    lignes = texte.split('\n')
    lignes_corrigees = []
    for i, ligne in enumerate(lignes):
        # Si la ligne n'est pas une nouvelle ligne de paragraphe et qu'elle n'est pas une liste à puce
        if i > 0 and ligne.strip() and not lignes[i-1].strip().endswith(('.', ':', '?', '!')) and not ligne.strip().startswith(('*', '-', '•')):
            # On la colle à la ligne précédente avec un espace
            if lignes_corrigees: # S'assurer que la liste n'est pas vide
                 lignes_corrigees[-1] = lignes_corrigees[-1].strip() + " " + ligne.strip()
            else:
                 lignes_corrigees.append(ligne)
        else:
            lignes_corrigees.append(ligne)
    texte = "\n".join(lignes_corrigees)
    
    # 4. Supprimer les espaces multiples et les tabulations
    texte = re.sub(r'[ \t]+', ' ', texte)
    
    # 5. Supprimer les lignes ne contenant que des espaces et les espaces en début/fin de ligne
    texte = "\n".join([line.strip() for line in texte.split('\n') if line.strip()])
    
    return texte.strip()

In [3]:
def convertir_tableau_en_markdown(tableau: list[list[str]]) -> str:
    """
    Convertit un tableau (liste de listes) en une chaîne de caractères au format Markdown.
    Version améliorée pour gérer les lignes vides et les cellules None.
    """
    # Filtrer les lignes vides ou invalides
    lignes_valides = [ligne for ligne in tableau if ligne and any(cell is not None for cell in ligne)]
    if not lignes_valides:
        return ""
    
    # Nettoyer les cellules individuelles (remplacer None par "" et supprimer les sauts de ligne)
    lignes_nettoyees = [
        [str(cell).replace('\n', ' ').strip() if cell is not None else "" for cell in ligne]
        for ligne in lignes_valides
    ]

    # Construire la table Markdown
    entetes = lignes_nettoyees[0]
    markdown = "| " + " | ".join(entetes) + " |\n"
    markdown += "| " + " | ".join(["---"] * len(entetes)) + " |\n"
    for ligne in lignes_nettoyees[1:]:
        # S'assurer que la ligne a le bon nombre de colonnes
        while len(ligne) < len(entetes):
            ligne.append("")
        markdown += "| " + " | ".join(ligne[:len(entetes)]) + " |\n"
        
    return markdown

In [4]:
def identifier_entetes_pieds_de_page(pdf: pdfplumber.PDF, pages_a_verifier: int = 3, zone_pourcentage: float = 0.15) -> tuple[set[str], set[str]]:
    """
    Analyse les premières pages d'un PDF pour identifier les en-têtes et pieds de page récurrents.

    Args:
        pdf: L'objet pdfplumber.PDF ouvert.
        pages_a_verifier: Le nombre de pages à comparer pour trouver des récurrences (min 2).
        zone_pourcentage: Le pourcentage en haut et en bas de la page à considérer comme zone potentielle.

    Returns:
        Un tuple contenant deux sets : (lignes_entetes_communes, lignes_pieds_de_page_communs).
    """
    if len(pdf.pages) < 2:
        return set(), set()

    # S'assurer de ne pas dépasser le nombre de pages disponibles
    pages_a_verifier = min(pages_a_verifier, len(pdf.pages))
    
    listes_entetes = []
    listes_pieds_de_page = []

    for i in range(pages_a_verifier):
        page = pdf.pages[i]
        hauteur_page = page.height
        largeur_page = page.width

        # Définir la "boîte" de la zone d'en-tête (ex: les 15% supérieurs de la page)
        zone_entete_bbox = (0, 0, largeur_page, hauteur_page * zone_pourcentage)
        # Définir la "boîte" de la zone de pied de page (ex: les 15% inférieurs)
        zone_pied_de_page_bbox = (0, hauteur_page * (1 - zone_pourcentage), largeur_page, hauteur_page)

        # Extraire le texte de ces zones
        entete_potentiel = page.crop(zone_entete_bbox).extract_text(x_tolerance=2)
        pied_de_page_potentiel = page.crop(zone_pied_de_page_bbox).extract_text(x_tolerance=2)
        
        # Nettoyer et normaliser le texte pour la comparaison
        # On enlève les chiffres pour que "Page 5" et "Page 6" soient considérés comme identiques
        if entete_potentiel:
            lignes = {re.sub(r'\d+', '', ligne).strip() for ligne in entete_potentiel.split('\n') if ligne.strip()}
            listes_entetes.append(lignes)

        if pied_de_page_potentiel:
            lignes = {re.sub(r'\d+', '', ligne).strip() for ligne in pied_de_page_potentiel.split('\n') if ligne.strip()}
            listes_pieds_de_page.append(lignes)

    # Trouver l'intersection : les lignes de texte présentes dans toutes les zones vérifiées
    entetes_communs = set.intersection(*listes_entetes) if listes_entetes else set()
    pieds_de_page_communs = set.intersection(*listes_pieds_de_page) if listes_pieds_de_page else set()
    
    return entetes_communs, pieds_de_page_communs

In [5]:
def traiter_pdf_avec_plumber_ameliore(chemin_pdf: Path) -> list[Document]:
    """
    (Version la plus robuste) Traite un PDF avec identification dynamique des en-têtes/pieds de page
    et inclut une option de débogage pour afficher une comparaison avant/après.
    """
    documents = []
    
    with pdfplumber.open(chemin_pdf) as pdf:
        entetes_a_supprimer, pieds_de_page_a_supprimer = identifier_entetes_pieds_de_page(pdf)

        for num_page, page in enumerate(pdf.pages):
            
            # --- CODE DE COMPARAISON (partie 1 : capturer "AVANT") ---
            texte_brut_avant_tout = ""
            # On ne fait cette opération que si le débogage est activé et pour les premières pages
            if AFFICHER_COMPARAISON_AVANT_APRES and num_page < PAGES_A_COMPARER_PAR_PDF:
                texte_brut_avant_tout = page.extract_text() or "[Aucun texte brut trouvé sur cette page]"
            # -------------------------------------------------------------

            lignes_de_texte = page.extract_text_lines(layout=True, strip=True)
            
            lignes_contenu_principal = []
            for ligne in lignes_de_texte:
                texte_ligne = ligne['text'].strip()
                texte_ligne_normalise = re.sub(r'\d+', '', texte_ligne).strip()
                
                if texte_ligne_normalise not in entetes_a_supprimer and texte_ligne_normalise not in pieds_de_page_a_supprimer:
                    lignes_contenu_principal.append(texte_ligne)
            
            texte_page_brut = "\n".join(lignes_contenu_principal)
            
            texte_page_nettoye = nettoyer_texte(texte_page_brut)
            
            markdown_tableaux = ""
            tableaux = page.extract_tables()
            if tableaux:
                for i, tableau in enumerate(tableaux):
                    if tableau and len(tableau) > 1:
                        md_table = convertir_tableau_en_markdown(tableau)
                        if md_table:
                            markdown_tableaux += f"\n\n--- Tableau {i+1} ---\n{md_table}"
            
            contenu_page_complet = f"{texte_page_nettoye}\n{markdown_tableaux}".strip()
            
            # --- CODE DE COMPARAISON (partie 2 : afficher "AVANT" et "APRÈS") ---
            if AFFICHER_COMPARAISON_AVANT_APRES and num_page < PAGES_A_COMPARER_PAR_PDF:
                print("\n" + "="*80)
                print(f"🔎 COMPARAISON POUR : '{chemin_pdf.name}', Page {num_page + 1}")
                print("="*80)
                
                print("\n--- AVANT NETTOYAGE (Texte brut de la page) ---\n")
                print(texte_brut_avant_tout)
                
                print("\n--- APRÈS NETTOYAGE (Contenu final qui sera stocké) ---\n")
                print(contenu_page_complet if contenu_page_complet else "[Page vide après nettoyage]")
                
                print("\n" + "="*80 + "\n")
            # -------------------------------------------------------------------------
            
            if contenu_page_complet:
                doc = Document(
                    page_content=contenu_page_complet,
                    metadata={
                        "source": str(chemin_pdf.name),
                        "page": num_page + 1
                    }
                )
                documents.append(doc)
            
    return documents

In [6]:
# Votre fonction `creer_documents_depuis_pdfs` est modifiée pour appeler la nouvelle fonction
def creer_documents_depuis_pdfs(chemin_repertoire: Path) -> list[Document]:
    """
    Scanne un répertoire, traite tous les PDF avec la méthode PLUMBER AMÉLIORÉE,
    et retourne une seule liste contenant tous les objets Document.
    """
    tous_les_docs = []
    fichiers_pdf = list(chemin_repertoire.glob('*.pdf'))

    if not fichiers_pdf:
        print(f"⚠️ Aucun fichier PDF trouvé dans : {chemin_repertoire}")
        return []

    for chemin_pdf in tqdm(fichiers_pdf, desc="Traitement de tous les PDF"):
        try:
            # ON APPELLE LA NOUVELLE FONCTION AMÉLIORÉE ICI
            docs_pdf = traiter_pdf_avec_plumber_ameliore(chemin_pdf)
            tous_les_docs.extend(docs_pdf)
        except Exception as e:
            print(f"❌ Erreur lors du traitement du fichier {chemin_pdf.name}: {e}")
            
    return tous_les_docs

# ==============================================================================
# === FIN DE LA SECTION AMÉLIORÉE ===
# ==============================================================================

In [7]:
print("\n--- ÉTAPE 1 : CRÉATION DES DOCUMENTS (AVEC NETTOYAGE AVANCÉ) ---")
# La logique principale ne change pas, elle utilise maintenant les fonctions améliorées
documents = creer_documents_depuis_pdfs(CHEMIN_DONNEES_BRUTES)

if documents:
    print(f"\n✅ Création des documents terminée. Nombre total de pages traitées : {len(documents)}.")
    print("\n--- Exemple de sortie nettoyée (Page 2 du premier PDF si elle existe) ---")
    # Afficher la page 2 pour voir l'effet de la suppression d'en-tête
    doc_a_afficher = documents[1] if len(documents) > 1 else documents[0]
    print(doc_a_afficher.page_content)
    print("\n--- Métadonnées associées ---")
    print(doc_a_afficher.metadata)
else:
    print("\n❌ La création des documents a échoué ou aucun texte n'a été extrait.")

if documents:
    # === ÉTAPE 2 : DÉCOUPAGE EN SEGMENTS (CHUNKING) ===
    print("\n--- ÉTAPE 2 : Découpage des documents en segments ---")
    diviseur_texte = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,       # Taille de chaque segment en caractères
        chunk_overlap=chunk_overlap,    # Chevauchement entre les segments
        length_function=len,
        separators=["\n\n", "\n", ". ", " ", ""] # Séparateurs très efficaces
    )
    segments_finaux = diviseur_texte.split_documents(documents)
    print(f"✅ Documents découpés en {len(segments_finaux)} segments.")
    print("\n--- Exemple de Segment ---")
    print(segments_finaux[0].page_content)
    print("\n--- Métadonnées du Segment ---")
    print(segments_finaux[0].metadata)

    # === ÉTAPE 3 : VECTORISATION (EMBEDDING) ===
    print(f"\n--- ÉTAPE 3 : Génération des vecteurs avec '{NOM_MODELE_SENTENCE_EMBEDDING}' ---")
    modele_embedding = SentenceTransformer(NOM_MODELE_SENTENCE_EMBEDDING, device='cuda')
    
    contenus_segments = [segment.page_content for segment in segments_finaux]
    embeddings_segments = modele_embedding.encode(contenus_segments, show_progress_bar=True, normalize_embeddings=True)
    print(f"✅ Vecteurs générés. Forme de la matrice de vecteurs : {embeddings_segments.shape}")

    # === ÉTAPES 4 & 5 : INDEXATION et SAUVEGARDE ===
    print("\n--- ÉTAPES 4 & 5 : Création de l'index FAISS et sauvegarde des artefacts ---")
    dimension_vecteurs = embeddings_segments.shape[1]
    index = faiss.IndexIDMap(faiss.IndexFlatIP(dimension_vecteurs))
    index.add_with_ids(embeddings_segments.astype('float32'), np.arange(len(segments_finaux)))

    CHEMIN_INDEX_FAISS = CHEMIN_DONNEES_TRAITEES / "documents.index"
    CHEMIN_SEGMENTS = CHEMIN_DONNEES_TRAITEES / "documents_segments.pkl"

    faiss.write_index(index, str(CHEMIN_INDEX_FAISS))
    with open(CHEMIN_SEGMENTS, "wb") as f:
        pickle.dump(segments_finaux, f)

    print(f"✅ Index sauvegardé dans : {CHEMIN_INDEX_FAISS}")
    print(f"✅ Segments (avec métadonnées) sauvegardés dans : {CHEMIN_SEGMENTS}")
    print("\n🎉 Pipeline d'ingestion terminé ! Le 'cerveau' de votre RAG est prêt. 🎉")

else:
    print("⚠️ Le découpage et la vectorisation sont ignorés car aucun document n'a été créé.")


--- ÉTAPE 1 : CRÉATION DES DOCUMENTS (AVEC NETTOYAGE AVANCÉ) ---


Traitement de tous les PDF: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:02<00:00,  3.11it/s]



✅ Création des documents terminée. Nombre total de pages traitées : 42.

--- Exemple de sortie nettoyée (Page 2 du premier PDF si elle existe) ---
Procédure DMGP-PR-05-01 I. Objet Cette procédure décrit les étapes de désignation des Tech Leads pour les six prochains mois au sein de l’équipe technique de IMPACTDEV. Son objectif est de garantir une désignation transparente, équitable et basée sur le vote des développeurs.
II. Domaine d’application Cette procédure est applicable à l’équipe technique de IMPACTDEV, incluant les développeurs des technologies React, Symfony et WordPress, et est dirigée par la direction technique.
III. Responsabilités Cette procédure est sous la responsabilité du responsable de la direction technique.
IV. Définitions et abréviations DT : Directeur technique Tech lead : (Technical Lead) est un expert technique qui guide l’équipe de développement en assurant la qualité technique des projets


--- Tableau 1 ---
|  | Procédure | DMGP-PR-05-01 |
| --- | --- | --- 

/home/elyes/miniconda3/envs/llm-dev-old/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

✅ Vecteurs générés. Forme de la matrice de vecteurs : (255, 384)

--- ÉTAPES 4 & 5 : Création de l'index FAISS et sauvegarde des artefacts ---
✅ Index sauvegardé dans : /home/elyes/stage/pdf-RAG-project/data/processed/documents.index
✅ Segments (avec métadonnées) sauvegardés dans : /home/elyes/stage/pdf-RAG-project/data/processed/documents_segments.pkl

🎉 Pipeline d'ingestion terminé ! Le 'cerveau' de votre RAG est prêt. 🎉
